# 🤖 GPT-5 Image Generation

**Module :** 01-Images-Foundation  
**Niveau :** 🟢 Débutant  
**Technologies :** GPT-5, Multimodal AI, OpenRouter  
**Durée estimée :** 20 minutes

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire
notebook_mode = "interactive"
skip_widgets = False
model_name = "openai/gpt-5"
max_tokens = 4000
temperature = 0.7

In [ ]:
# Setup et génération GPT-5 avec capacités images
print("🤖 GPT-5 Image Generation Setup")
print("Configuration multimodale en cours...")

# Import et configuration
import os
import requests
from datetime import datetime

api_key = os.getenv('OPENROUTER_API_KEY')
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

print(f"✅ GPT-5 configuré - {datetime.now().strftime('%H:%M:%S')}")